<a href="https://colab.research.google.com/github/gaallmin/Prediction-of-Agricultural-Product-prices/blob/yeji/%EA%B2%B0%EC%B8%A1%EC%B9%98_%EB%8C%80%EC%B2%B4_meta%ED%8F%AC%ED%95%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive/')
%cd /content/drive/MyDrive/Dacon

Mounted at /content/drive/
/content/drive/MyDrive/Dacon


In [5]:
import pandas as pd

train_file_path = './train/train.csv'
market_file_path = './train/meta/TRAIN_전국도매_2018-2021.csv'
sanjae_file_path = './train/meta/TRAIN_산지공판장_2018-2021.csv'

train_df = pd.read_csv(train_file_path)
market_df = pd.read_csv(market_file_path)
sanjae_df = pd.read_csv(sanjae_file_path)

In [6]:
import re

# '시장명'이 "전국도매시장"인 경우만 필터링
market_df = market_df[market_df['시장명'].str.contains('전국도매시장')]

# '공판장명'이 "전국농협공판장"인 경우만 필터링
sanjae_df = sanjae_df[sanjae_df['공판장명'].str.contains('전국농협공판장')]

sanjae_df.rename(columns = {'등급명': '등급'}, inplace = True)

# 컬럼명 변경 (도매 데이터의 컬럼들에 '(도매)', 산지 데이터의 컬럼들에 '(산지)' 추가)
market_columns = market_df.columns.difference(['시점', '품목명', '품종명'])  # 키 컬럼 제외
market_renamed = {col: col + ' (도매)' for col in market_columns}

sanjae_columns = sanjae_df.columns.difference(['시점', '품목명', '품종명', '등급'])  # 키 컬럼 제외
sanjae_renamed = {col: col + ' (산지)' for col in sanjae_columns}

# 컬럼명 변경 적용
market_df.rename(columns={**market_renamed}, inplace=True)
sanjae_df.rename(columns={**sanjae_renamed}, inplace=True)

# '시점', '품목명', '품종명' 기준으로 market_df와 left join, '등급명'은 train_df에만 존재
merged_df = pd.merge(train_df, market_df, how='left', on=['시점', '품목명', '품종명'])

# '시점', '품목명', '품종명', '등급명' 기준으로 sanjae_df와 left join
merged_sanjae_df = pd.merge(merged_df, sanjae_df, how='left', on=['시점', '품목명', '품종명', '등급'])

# '거래단위'에서 'kg'을 제외한 숫자만 추출하여 새로운 컬럼 '거래단위_숫자' 생성
merged_sanjae_df['거래단위_숫자'] = merged_sanjae_df['거래단위'].apply(lambda x: float(re.sub(r'[^0-9.]', ' ', x)))

# '거래단위'에서 '5톤트럭'과 '8톤트럭' 처리
def process_truck_units(unit):
    if '5톤트럭' in unit:
        return 5000
    elif '8톤트럭' in unit:
        return 8000
    else:
        return float(re.sub(r'[^0-9.]', '', unit))  # 그 외의 경우 숫자만 추출

merged_sanjae_df['거래단위_숫자'] = merged_sanjae_df['거래단위'].apply(process_truck_units)


# '평균가(원/kg)'와 '거래단위_숫자'를 곱해서 '거래금액(도매)' 및 '거래금액(산지)' 컬럼에 저장
merged_sanjae_df['거래금액(도매)'] = merged_sanjae_df['평균가(원/kg) (도매)'] * merged_sanjae_df['거래단위_숫자']
merged_sanjae_df['거래금액(산지)'] = merged_sanjae_df['평균가(원/kg) (산지)'] * merged_sanjae_df['거래단위_숫자']

# '거래금액(도매)'와 '거래금액(산지)'에서 값이 없거나 0인 경우 0으로 처리
merged_sanjae_df['거래금액(도매)'] = merged_sanjae_df['거래금액(도매)'].fillna(0)
merged_sanjae_df['거래금액(도매)'] = merged_sanjae_df['거래금액(도매)'].apply(lambda x: 0 if x == 0 or pd.isna(x) else x)

merged_sanjae_df['거래금액(산지)'] = merged_sanjae_df['거래금액(산지)'].fillna(0)
merged_sanjae_df['거래금액(산지)'] = merged_sanjae_df['거래금액(산지)'].apply(lambda x: 0 if x == 0 or pd.isna(x) else x)


<ipython-input-6-23997021a5d0>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sanjae_df.rename(columns = {'등급명': '등급'}, inplace = True)
<ipython-input-6-23997021a5d0>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  market_df.rename(columns={**market_renamed}, inplace=True)
<ipython-input-6-23997021a5d0>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sanjae_df.rename(columns={**sanjae_renamed}, inplace=True)


In [7]:
import pandas as pd

# 품종 쌍과 거래단위를 설정
product_pairs = [
    ('양파', '20키로', '양파', '10키로'),
    ('양파', '12키로', '양파(햇)', '12키로'),
    ('양파', '15키로', '양파(햇)', '15키로'),
    ('양파', '20키로', '양파(햇)', '20키로'),
    ('양파', '15키로', '저장양파', '15키로'),
    ('양파', '12키로', '저장양파', '12키로'),
    ('양파', '20키로', '저장양파', '20키로'),
    ('양파', '12키로', '조생양파', '12키로'),
    ('양파', '15키로', '조생양파', '15키로'),
    ('감자 수미', '20키로상자', '감자', '20키로상자'),
    ('감자 수미', '20키로상자', '감자 대지', '20키로상자'),
    ('감자 수미', '20키로상자', '감자 두백', '20키로상자'),
    ('감자 수미', '20키로상자', '감자 수미(저장)', '20키로상자'),
    ('감자 수미', '20키로상자', '감자 수미(햇)', '20키로상자'),
    ('감자 수미', '20키로상자', '감자 수입', '23키로상자'),
    ('감자 수미', '20키로상자', '감자 조풍', '20키로상자'),
    ('감자 수미', '20키로상자', '홍감자', '10키로상자'),
    ('화건', '30 kg', '햇산화건', '30 kg'),
    ('양건', '30 kg', '햇산양건', '30 kg'),
    ('실파', '20키로', '쪽파', '10키로상자'),
    ('쪽파', '10키로상자', '대파 수입', '10키로묶음'),
    ('쪽파', '10키로상자', '대파(일반)', '10키로묶음'),
    ('쪽파', '10키로상자', '대파(일반)', '1키로단'),
    ('무', '20키로상자', '무', '18키로상자(비닐포)'),
    ('무', '20키로상자', '다발무', '5톤트럭'),
    ('신고', '10 개', '원황', '10 개'),
    ('봄동배추', '15키로상자', '절임배추', '20키로상자'),
    ('쌈배추', '8키로상자', '쌈배추', '1키로상자'),
    ('얼갈이배추', '4키로상자', '얼갈이배추', '8키로상자'),
    ('후지', '10 개', '쓰가루', '10 개')
]

# 가격 컬럼 리스트
price_columns = ['평균가격(원)', '평년 평균가격(원)', '거래금액(도매)', '거래금액(산지)']

# 평균 비율을 계산하는 함수 (거래단위를 포함)
def calculate_mean_ratio(df1, df2, price_col):
    # '시점', '등급', '거래단위'를 기준으로 inner join
    merged_df = pd.merge(df1[['시점', price_col, '등급', '거래단위']],
                         df2[['시점', price_col, '등급', '거래단위']],
                         on=['시점', '등급', '거래단위'], suffixes=('_1', '_2'), how='inner')

    # 두 품종의 가격이 모두 0이 아닌 경우만 사용
    valid_rows = merged_df[(merged_df[f'{price_col}_1'] != 0) & (merged_df[f'{price_col}_2'] != 0)]

    # 비율 계산 및 평균 비율 구하기
    valid_rows['비율'] = valid_rows[f'{price_col}_1'] / valid_rows[f'{price_col}_2']
    mean_ratio = valid_rows['비율'].mean()

    return mean_ratio

# 0인 값을 대체하는 함수 (평균 비율을 사용)
def replace_zero_in_train(row, mean_ratio, product_1, unit_1, product_2, unit_2, price_col):
    if row['품종명'] == product_1 and row['거래단위'] == unit_1:
        # 같은 시점, 등급, 거래단위의 다른 품종 가격 가져오기
        matching_row = merged_sanjae_df[(merged_sanjae_df['시점'] == row['시점']) &
                                (merged_sanjae_df['품종명'] == product_2) &
                                (merged_sanjae_df['등급'] == row['등급']) &
                                (merged_sanjae_df['거래단위'] == unit_2)]
        if not matching_row.empty:
            other_price = matching_row[price_col].values[0]
            if row[price_col] == 0 and other_price != 0:
                row[price_col] = other_price * mean_ratio
    elif row['품종명'] == product_2 and row['거래단위'] == unit_2:
        # 같은 시점, 등급, 거래단위의 첫 번째 품종 가격 가져오기
        matching_row = merged_sanjae_df[(merged_sanjae_df['시점'] == row['시점']) &
                                (merged_sanjae_df['품종명'] == product_1) &
                                (merged_sanjae_df['등급'] == row['등급']) &
                                (merged_sanjae_df['거래단위'] == unit_1)]
        if not matching_row.empty:
            other_price = matching_row[price_col].values[0]
            if row[price_col] == 0 and other_price != 0:
                row[price_col] = other_price / mean_ratio
    return row

# 각 품종 쌍과 가격 컬럼에 대해 처리 적용
for product_1, unit_1, product_2, unit_2 in product_pairs:
    for price_col in price_columns:
        # 해당 품종과 거래단위 필터링
        df1 = merged_sanjae_df[(merged_sanjae_df['품종명'] == product_1) & (merged_sanjae_df['거래단위'] == unit_1)]
        df2 = merged_sanjae_df[(merged_sanjae_df['품종명'] == product_2) & (merged_sanjae_df['거래단위'] == unit_2)]

        # 평균 비율 계산
        mean_ratio = calculate_mean_ratio(df1, df2, price_col)
        print(f"{product_1}({unit_1}) - {product_2}({unit_2})의 {price_col}에 대한 평균 비율: {mean_ratio}")

        # 0인 값을 평균 비율로 대체
        merged_sanjae_df = merged_sanjae_df.apply(replace_zero_in_train, axis=1, mean_ratio=mean_ratio,
                                  product_1=product_1, unit_1=unit_1,
                                  product_2=product_2, unit_2=unit_2,
                                  price_col=price_col)


양파(20키로) - 양파(10키로)의 평균가격(원)에 대한 평균 비율: nan
양파(20키로) - 양파(10키로)의 평년 평균가격(원)에 대한 평균 비율: nan
양파(20키로) - 양파(10키로)의 거래금액(도매)에 대한 평균 비율: nan
양파(20키로) - 양파(10키로)의 거래금액(산지)에 대한 평균 비율: nan


<ipython-input-7-7d7278b01e1d>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_rows['비율'] = valid_rows[f'{price_col}_1'] / valid_rows[f'{price_col}_2']


양파(12키로) - 양파(햇)(12키로)의 평균가격(원)에 대한 평균 비율: 1.0126329222765227
양파(12키로) - 양파(햇)(12키로)의 평년 평균가격(원)에 대한 평균 비율: nan
양파(12키로) - 양파(햇)(12키로)의 거래금액(도매)에 대한 평균 비율: nan
양파(12키로) - 양파(햇)(12키로)의 거래금액(산지)에 대한 평균 비율: nan


<ipython-input-7-7d7278b01e1d>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_rows['비율'] = valid_rows[f'{price_col}_1'] / valid_rows[f'{price_col}_2']


양파(15키로) - 양파(햇)(15키로)의 평균가격(원)에 대한 평균 비율: 1.0011656821945343
양파(15키로) - 양파(햇)(15키로)의 평년 평균가격(원)에 대한 평균 비율: nan
양파(15키로) - 양파(햇)(15키로)의 거래금액(도매)에 대한 평균 비율: nan
양파(15키로) - 양파(햇)(15키로)의 거래금액(산지)에 대한 평균 비율: nan


...

신고(10 개) - 원황(10 개)의 거래금액(도매)에 대한 평균 비율: 1.5413807055690891
신고(10 개) - 원황(10 개)의 거래금액(산지)에 대한 평균 비율: nan
봄동배추(15키로상자) - 절임배추(20키로상자)의 평균가격(원)에 대한 평균 비율: nan
봄동배추(15키로상자) - 절임배추(20키로상자)의 평년 평균가격(원)에 대한 평균 비율: nan
봄동배추(15키로상자) - 절임배추(20키로상자)의 거래금액(도매)에 대한 평균 비율: nan
봄동배추(15키로상자) - 절임배추(20키로상자)의 거래금액(산지)에 대한 평균 비율: nan
쌈배추(8키로상자) - 쌈배추(1키로상자)의 평균가격(원)에 대한 평균 비율: nan
쌈배추(8키로상자) - 쌈배추(1키로상자)의 평년 평균가격(원)에 대한 평균 비율: nan
쌈배추(8키로상자) - 쌈배추(1키로상자)의 거래금액(도매)에 대한 평균 비율: nan
쌈배추(8키로상자) - 쌈배추(1키로상자)의 거래금액(산지)에 대한 평균 비율: nan
얼갈이배추(4키로상자) - 얼갈이배추(8키로상자)의 평균가격(원)에 대한 평균 비율: nan
얼갈이배추(4키로상자) - 얼갈이배추(8키로상자)의 평년 평균가격(원)에 대한 평균 비율: nan
얼갈이배추(4키로상자) - 얼갈이배추(8키로상자)의 거래금액(도매)에 대한 평균 비율: nan
얼갈이배추(4키로상자) - 얼갈이배추(8키로상자)의 거래금액(산지)에 대한 평균 비율: nan


<ipython-input-7-7d7278b01e1d>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_rows['비율'] = valid_rows[f'{price_col}_1'] / valid_rows[f'{price_col}_2']


후지(10 개) - 쓰가루(10 개)의 평균가격(원)에 대한 평균 비율: 1.402595682437155


<ipython-input-7-7d7278b01e1d>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_rows['비율'] = valid_rows[f'{price_col}_1'] / valid_rows[f'{price_col}_2']


후지(10 개) - 쓰가루(10 개)의 평년 평균가격(원)에 대한 평균 비율: 1.5405378902819933
후지(10 개) - 쓰가루(10 개)의 거래금액(도매)에 대한 평균 비율: nan
후지(10 개) - 쓰가루(10 개)의 거래금액(산지)에 대한 평균 비율: nan


In [8]:
import pandas as pd
import re

# 품종 쌍과 거래단위 설정
product_pairs = [
    ('다발무', '5톤트럭', '다발무', '10키로'),
    ('다발무', '5톤트럭', '다발무', '5000키로'),
    ('다발무', '5톤트럭', '다발무', '1000키로'),
    ('다발무', '5톤트럭', '다발무', '10키로'),
    ('다발무', '5톤트럭', '다발무', '8톤트럭')
]

# 가격 및 거래단위 컬럼 리스트
price_columns = ['평균가격(원)', '평년 평균가격(원)', '거래금액(도매)', '거래금액(산지)']
unit_column = '거래단위_숫자'

# 0인 값을 대체하는 함수
def replace_zero_with_unit_price(row, product_1, unit_1, product_2, unit_2, price_col):
    if row['품종명'] == product_1 and row['거래단위'] == unit_1:
        # 같은 시점, 등급의 다른 품종과 거래단위의 가격 가져오기
        matching_row = merged_sanjae_df[(merged_sanjae_df['시점'] == row['시점']) &
                                (merged_sanjae_df['품종명'] == product_2) &
                                (merged_sanjae_df['등급'] == row['등급']) &
                                (merged_sanjae_df['거래단위'] == unit_2)]
        if not matching_row.empty:
            other_price = matching_row[price_col].values[0]
            if row[price_col] == 0 and other_price != 0:
                # 첫 번째 품종의 가격이 0이면 두 번째 품종의 가격에 거래단위_숫자를 곱해 단위가격으로 대체
                row[price_col] = other_price * row['거래단위_숫자'] / matching_row['거래단위_숫자'].values[0]
    elif row['품종명'] == product_2 and row['거래단위'] == unit_2:
        # 같은 시점, 등급의 첫 번째 품종과 거래단위의 가격 가져오기
        matching_row = merged_sanjae_df[(merged_sanjae_df['시점'] == row['시점']) &
                                (merged_sanjae_df['품종명'] == product_1) &
                                (merged_sanjae_df['등급'] == row['등급']) &
                                (merged_sanjae_df['거래단위'] == unit_1)]
        if not matching_row.empty:
            other_price = matching_row[price_col].values[0]
            if row[price_col] == 0 and other_price != 0:
                # 두 번째 품종의 가격이 0이면 첫 번째 품종의 단위가격을 대체
                row[price_col] = (other_price / matching_row['거래단위_숫자'].values[0]) * row['거래단위_숫자']
    return row

# 각 품종 쌍과 가격 컬럼에 대해 처리 적용
for product_1, unit_1, product_2, unit_2 in product_pairs:
    for price_col in price_columns:
        # 0인 값을 단위가격으로 대체
        merged_sanjae_df = merged_sanjae_df.apply(replace_zero_with_unit_price, axis=1,
                                  product_1=product_1, unit_1=unit_1,
                                  product_2=product_2, unit_2=unit_2, price_col=price_col)


In [9]:
merged_sanjae_df['평균가격(원)'] = merged_sanjae_df['평균가격(원)'].fillna(0)
merged_sanjae_df['평균가격(원)'] = merged_sanjae_df['평균가격(원)'].apply(lambda x: 0 if x == 0 or pd.isna(x) else x)

merged_sanjae_df['평년 평균가격(원)'] = merged_sanjae_df['평년 평균가격(원)'].fillna(0)
merged_sanjae_df['평년 평균가격(원)'] = merged_sanjae_df['평년 평균가격(원)'].apply(lambda x: 0 if x == 0 or pd.isna(x) else x)

merged_sanjae_df['거래금액(도매)'] = merged_sanjae_df['거래금액(도매)'].fillna(0)
merged_sanjae_df['거래금액(도매)'] = merged_sanjae_df['거래금액(도매)'].apply(lambda x: 0 if x == 0 or pd.isna(x) else x)

merged_sanjae_df['거래금액(산지)'] = merged_sanjae_df['거래금액(산지)'].fillna(0)
merged_sanjae_df['거래금액(산지)'] = merged_sanjae_df['거래금액(산지)'].apply(lambda x: 0 if x == 0 or pd.isna(x) else x)

In [16]:
# 필요한 열만 선택하여 새로운 데이터프레임 생성
columns_to_keep = ['시점', '품목명', '품종명', '등급', '거래단위', '평년 평균가격(원)', '평균가격(원)', '거래금액(도매)', '거래금액(산지)']
filtered_sanjae_df = merged_sanjae_df[columns_to_keep]

In [17]:
# 수정된 데이터를 새로운 CSV 파일로 저장 (UTF-8-SIG 인코딩 사용)
output_file_path = './train_meta_v1.csv'
filtered_sanjae_df.to_csv(output_file_path, index=False, encoding='utf-8-sig')

In [18]:
import pandas as pd

# 데이터 불러오기
train_meta_v1_df = pd.read_csv('./train_meta_v1.csv')

# 필터링할 조건 설정 (행별로 조건을 각각 명시)
condition_1 = (
    (train_meta_v1_df['품목명'] == '건고추') &
    (train_meta_v1_df['품종명'] == '화건') &
    (train_meta_v1_df['거래단위'] == '30 kg') &
    (train_meta_v1_df['등급'] == '상품')
)

condition_2 = (
    (train_meta_v1_df['품목명'] == '사과') &
    (train_meta_v1_df['품종명'].isin(['홍로', '후지'])) &
    (train_meta_v1_df['거래단위'] == '10 개') &
    (train_meta_v1_df['등급'] == '상품')
)

condition_3 = (
    (train_meta_v1_df['품목명'] == '감자') &
    (train_meta_v1_df['품종명'] == '감자 수미') &
    (train_meta_v1_df['거래단위'] == '20키로상자') &
    (train_meta_v1_df['등급'] == '상')
)

condition_4 = (
    (train_meta_v1_df['품목명'] == '배') &
    (train_meta_v1_df['품종명'] == '신고') &
    (train_meta_v1_df['거래단위'] == '10 개') &
    (train_meta_v1_df['등급'] == '상품')
)

condition_5 = (
    (train_meta_v1_df['품목명'] == '깐마늘(국산)') &
    (train_meta_v1_df['품종명'] == '깐마늘(국산)') &
    (train_meta_v1_df['거래단위'] == '20 kg') &
    (train_meta_v1_df['등급'] == '상품')
)

condition_6 = (
    (train_meta_v1_df['품목명'] == '무') &
    (train_meta_v1_df['품종명'] == '무') &
    (train_meta_v1_df['거래단위'] == '20키로상자') &
    (train_meta_v1_df['등급'] == '상품')
)

condition_7 = (
    (train_meta_v1_df['품목명'] == '상추') &
    (train_meta_v1_df['품종명'] == '청') &
    (train_meta_v1_df['거래단위'] == '100 g') &
    (train_meta_v1_df['등급'] == '상품')
)

condition_8 = (
    (train_meta_v1_df['품목명'] == '배추') &
    (train_meta_v1_df['품종명'] == '배추') &
    (train_meta_v1_df['거래단위'] == '10키로망대') &
    (train_meta_v1_df['등급'] == '상')
)

condition_9 = (
    (train_meta_v1_df['품목명'] == '양파') &
    (train_meta_v1_df['품종명'] == '양파') &
    (train_meta_v1_df['거래단위'] == '1키로') &
    (train_meta_v1_df['등급'] == '상')
)

condition_10 = (
    (train_meta_v1_df['품목명'] == '대파') &
    (train_meta_v1_df['품종명'] == '대파(일반)') &
    (train_meta_v1_df['거래단위'] == '1키로단') &
    (train_meta_v1_df['등급'] == '상')
)

# 모든 조건을 합치기 (OR로 연결)
final_condition = condition_1 | condition_2 | condition_3 | condition_4 | condition_5 | condition_6 | condition_7 | condition_8 | condition_9 | condition_10

# 조건에 맞는 데이터 추출
filtered_meta_df = train_meta_v1_df[final_condition]

In [19]:
# 수정된 데이터를 새로운 CSV 파일로 저장 (UTF-8-SIG 인코딩 사용)
output_file_path = './train_meta_v1_s.csv'
filtered_meta_df.to_csv(output_file_path, index=False, encoding='utf-8-sig')